In [1]:
from openai import OpenAI
import os
import json
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import instructor # I'm not sure if this will actually be needed...
from pydantic import BaseModel, Field
from typing import List, Dict

In [2]:
load_dotenv('../.env')

openai_api_key = os.getenv('OPENAI_API_KEY')

# Test OpenAI API...

client = OpenAI(
  api_key=openai_api_key,  # this is also the default, it can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Can you state the name of a real species of bird? Only reply with the species name.",
        }
    ],
    model="gpt-4o-mini",
)

result = chat_completion.choices[0].message.content
print(result)

Pica pica


# Plan of action

The ideal plan will be to:
- Download FASTQ files
- Download metadata

With a bit of exploration, I have come to the realisation that some datasets will have both human and mouse samples (for example). Therefore, guiding the type of Kallisto analysis that I do will be important...

What do I specifically need to do?
- I do honestly think downloading the FASTQ files is straightforward:
- Download metadata, then identify the SRR ID (or whatever it's called - EDIT - turns out this is not so straightforward. There's either a knowledge gap in me, or I will need an LLM here)
- prefetch (SRR)
- fasterq dump (x)
- kallisto quant (x)

In between I can use an LLM call to just say "did this work"?

In [1]:
import subprocess

# Define the path to the R script
r_script_path = "./RScript_GetGEOMetadata.r"  # Ensure the path is correct and script is executable

# Define the GEO accession you want to use
geo_accession = "GSE273561"  # Replace with the actual GEO accession number

# Define output directory

outdir = "../Testing/GSE273561"

# Construct the command to run the R script with the GEO accession as an argument
command = ["Rscript", r_script_path, "-g", geo_accession, "-o", outdir]

# Call the R script using subprocess
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("R script output:")
    print(result.stdout)  # This will print the standard output from the R script
except subprocess.CalledProcessError as e:
    print("Error running R script:")
    print(e.stderr)  # This will print any error messages from the R script

R script output:
Output directory exists: ../Testing/GSE273561 
Retrieving metadata for GEO accession: GSE273561 
Saving metadata to: ../Testing/GSE273561/GSE273561-GPL24247_series_matrix_metadata.csv 
Saving metadata to: ../Testing/GSE273561/GSE273561-GPL24676_series_matrix_metadata.csv 
Metadata saved successfully!

